In [70]:
import pandas as pd
pd.set_option('display.max_rows' , 200)

In [71]:
company_name = 'TCS'

In [72]:
share_price_in = 1000000;

In [73]:
def calculate_growth(initial , final) :
    if not initial :
        return 1
    return round(((final - initial) / initial) * 100,2)

In [74]:
def get_share_price_in(price) :
    return round((price / share_price_in),2)

    

In [75]:
def vertical_analysis(item , divider = 12.43) :
    return abs(round((item/divider) * 100 ,2))
    

In [76]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [77]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [78]:
income_statement.rename(columns={company_name+'_income-statement_Annual_As_Originally_Reported' : 'items'} , inplace=True)
balance_sheet.rename(columns={company_name+'_balance-sheet_Annual_As_Originally_Reported' : 'items'} , inplace=True)
cashflow_statement.rename(columns={company_name+'_cash-flow_Annual_As_Originally_Reported' : 'items'} , inplace=True)

In [79]:
basic_eps_filter = income_statement['items'].eq('Basic EPS')
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [80]:
WASO_filter = income_statement['items'].eq('Basic Weighted Average Shares Outstanding') | income_statement['items'].eq('Diluted Weighted Average Shares Outstanding') | income_statement['items'].eq('Basic WASO') | income_statement['items'].eq('Diluted WASO')
waso_pos = income_statement[WASO_filter].index

In [81]:
income_statement.dropna(how='all',subset=income_statement.columns[1:], inplace=True)
balance_sheet.dropna(how='all',subset=balance_sheet.columns[1:], inplace=True)
cashflow_statement.dropna(how='all',subset=cashflow_statement.columns[1:], inplace=True)
income_statement.fillna(0 , inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [82]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [83]:
temp_df = temp_df.apply(get_share_price_in)

In [84]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [85]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [86]:
income_statement['items'] = income_statement['items'].apply(lambda x: x.strip())

In [87]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [88]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [89]:
vertical_analysis_is = pd.DataFrame(columns=income_statement.columns)
vertical_analysis_is['items'] = income_statement.loc[0:basic_eps_pos - 1,'items'].copy()


In [90]:
total_revenue_filter = income_statement['items'].eq('Total Revenue')
total_revenue = income_statement.loc[total_revenue_filter,start_col:]

In [91]:
for column in vertical_analysis_is.columns[1:] : 
    vertical_analysis_is[column] = income_statement.loc[0:basic_eps_pos - 1,column].apply(vertical_analysis,divider=total_revenue[column])
    